1. Реализовать функцию свертки (специфицировать размер и количество фильтров, входной тензор, stride, ...)
$$O[m][x][y] = \sum_{i=0}^{R-1}\sum_{j=0}^{S-1}\sum_{k=0}^{C-1}I[k][x+i][y+j] * W[m][k][i][j]$$
2. Написать фунцию реализующую сверточный слой через im2col. Сделать проверку результата с помощью прямой реализации свертки.
3. Специфицировать и написать функцию реализующиую Depthwise-separable свертку.

In [1]:
import numpy as np
import torch
from tqdm import tqdm

## 1.

In [68]:
def convolution(input_tensor, matrix_weight, stride=1):
    C, H_in, W_in = input_tensor.shape
    
    M, C, R, S = matrix_weight.shape
    
    H_out = (H_in - R) // stride + 1
    W_out = (W_in - S) // stride + 1
    
    output = np.zeros((M, H_out, W_out))
    
    for m in range(M):
        for x in range(H_out):
            for y in range(W_out):
                for i in range(R):
                    for j in range(S):
                        for c in range(C):
                            output[m, x, y] += input_tensor[c, x * stride + i, y * stride + j] * matrix_weight[m, c, i, j]
    
    return output

## 2.

In [69]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    C, H, W = input_data.shape
    H_out = (H + 2 * pad - filter_h) // stride + 1
    W_out = (W + 2 * pad - filter_w) // stride + 1

    Input_padded = np.pad(input_data, [(0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((C * filter_h * filter_w, H_out * W_out))

    i = 0
    for y in range(0, H + 2 * pad - filter_h + 1, stride):
        for x in range(0, W + 2 * pad - filter_w + 1, stride):
            patch = Input_padded[:, y:y + filter_h, x:x + filter_w].reshape(C * filter_h * filter_w)
            col[:, i] = patch
            i += 1
    
    return col


def convolution_im2col(input_tensor, matrix_weight, stride=1, pad=0):
    C, H_in, W_in = input_tensor.shape
    M, C, R, S = matrix_weight.shape
    H_out = (H_in + 2 * pad - R) // stride + 1
    W_out = (W_in + 2 * pad - S) // stride + 1

    col = im2col(input_tensor, R, S, stride, pad)
    col_W = matrix_weight.reshape(M, -1)
    out = np.dot(col_W, col)

    out = out.reshape(M, H_out, W_out)

    return out

## Проверка

In [100]:
size = 100
for stride in tqdm([1,2,3,4,5]):
    for channels in [1,2,3,4,5]:
        for output_channels in [1,2,3,4,5]:
            for filter_size in [3, 5, 7, 9]:
                input_tensor = np.random.rand(channels, size, size) 
                matrix_weight = np.random.rand(output_channels, channels, filter_size, filter_size)
                conv = convolution(input_tensor, matrix_weight, stride)
                conv_im2col = convolution_im2col(input_tensor, matrix_weight, stride)
                conv_torch = torch.nn.functional.conv2d(torch.Tensor(input_tensor), torch.Tensor(matrix_weight), stride=stride)
                assert np.allclose(conv, conv_torch)
                assert np.allclose(conv_im2col, conv_torch)
print("Тест пройден успешно!")            

100%|██████████| 5/5 [10:32<00:00, 126.53s/it]

Тест пройден успешно!
